In [3]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import dask
from dask.diagnostics import ProgressBar
import os

In [2]:
tmp_data_path = '../MA_data/data/tmp'
data_path = '../MA_data/data'

s_year = 1997
e_year = 2020


In [4]:
sdc_tnic = pd.read_pickle(tmp_data_path+f"/sdc_tnic_{s_year}_{e_year}")

In [5]:
tnic = pd.read_pickle(tmp_data_path+f"/tnic2_{s_year-1}_{e_year-1}")


In [6]:
with open(tmp_data_path+f"/tnic_info_3_pairs_{s_year-1}_{e_year-1}", 'rb') as f:
    gvkey_lsts, key_ind_maps , ind_key_maps = pickle.load(f)

In [14]:
cpu_cores = os.cpu_count()

# Processing

In [30]:
def find_top_peer_lst(tnic, gvkey, year, k, thres):
    '''
    Obtain top peers 
    tnic has: year, gvkey1, gvkey2, score, rank
    '''
    top_peers_subdf =  tnic[(tnic.year == year)]
    top_peers_subdf = top_peers_subdf[(top_peers_subdf.gvkey1 == gvkey)] 
    top_peers = top_peers_subdf[(top_peers_subdf['score'] >= thres)]
    
    if top_peers.shape[0] < k:
        top_peers_10 = top_peers_subdf[(top_peers_subdf['yearly_rank'] <= k)]
        if top_peers.shape[0] < top_peers_10.shape[0]:
            top_peers = top_peers_10
    
    top_peers_lst = top_peers.gvkey2.tolist()
    return top_peers_lst

In [31]:
@dask.delayed
def create_top_peer_map(tnic, year, gvkey_lsts, k=10, thres=0.4):
    y = year
    year_lst = {}
    #print("processing year ", y)
    for gvkey1 in gvkey_lsts[y]:
        year_lst[gvkey1] = find_top_peer_lst(tnic, gvkey1, y, k, thres)
    with open(tmp_data_path+f"/top_peers_{y}", 'wb') as f:
        pickle.dump(year_lst, f)

In [32]:
update_tasks_s = [create_top_peer_map(tnic, year, gvkey_lsts) for year in range(1996, 2019+1)]

In [33]:
with ProgressBar():
    dask.compute(update_tasks_s, num_workers = cpu_cores)

[                                        ] | 0% Completed |  9hr  6min 10.7s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[########################################] | 100% Completed | 12hr 55min 36.1s
